In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
batch_size = 64
learning_rate = 0.001
num_epochs = 10
input_size = 28 * 28  # MNIST images are 28x28 pixels
hidden_size = 128
num_classes = 10  # 10 digits (0-9)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST mean and std
])

In [5]:
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    transform=transform,
    download=True
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    transform=transform
)

# Create data loaders
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)


100%|██████████| 9.91M/9.91M [00:00<00:00, 42.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.19MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 10.4MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.14MB/s]


In [6]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

In [7]:
model = MLP(input_size, hidden_size, num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [9]:
def train():
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        if (i+1) % 100 == 0:
            print(f'Batch [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    return epoch_loss, accuracy

In [10]:
def test():
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [11]:
print(f"Training on {device}")
for epoch in range(num_epochs):
    train_loss, train_acc = train()
    test_acc = test()

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, '
          f'Train Accuracy: {train_acc:.2f}%, '
          f'Test Accuracy: {test_acc:.2f}%')

# Save the model
torch.save(model.state_dict(), 'mnist_mlp.pth')
print('Model saved to mnist_mlp.pth')

Training on cpu
Batch [100/938], Loss: 0.3245
Batch [200/938], Loss: 0.2276
Batch [300/938], Loss: 0.1827
Batch [400/938], Loss: 0.1443
Batch [500/938], Loss: 0.3945
Batch [600/938], Loss: 0.2700
Batch [700/938], Loss: 0.3124
Batch [800/938], Loss: 0.0445
Batch [900/938], Loss: 0.2850
Epoch [1/10], Train Loss: 0.2539, Train Accuracy: 92.46%, Test Accuracy: 96.22%
Batch [100/938], Loss: 0.1068
Batch [200/938], Loss: 0.1751
Batch [300/938], Loss: 0.0483
Batch [400/938], Loss: 0.1308
Batch [500/938], Loss: 0.0809
Batch [600/938], Loss: 0.0582
Batch [700/938], Loss: 0.1647
Batch [800/938], Loss: 0.1128
Batch [900/938], Loss: 0.0497
Epoch [2/10], Train Loss: 0.1041, Train Accuracy: 96.76%, Test Accuracy: 96.54%
Batch [100/938], Loss: 0.0306
Batch [200/938], Loss: 0.0737
Batch [300/938], Loss: 0.1546
Batch [400/938], Loss: 0.0768
Batch [500/938], Loss: 0.0468
Batch [600/938], Loss: 0.0102
Batch [700/938], Loss: 0.0427
Batch [800/938], Loss: 0.1526
Batch [900/938], Loss: 0.0640
Epoch [3/10], 